## Задачи
- Посчитать, сколько книг вышло после 1 января 2000 года
- Для каждой книги посчитать количество обзоров и среднюю оценку
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так брошюры будут исключены из анализа
- Определить автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками
- Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

## Компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Моя первая задача как аналитика — проанализировать базу данных.

In [1]:
import pandas as pd
from sqlalchemy import text, create_engine

db_config = {'user': '', 
'pwd': '', 
'host': '',
'port': ,
'db': ''} 
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

engine = create_engine(connection_string, connect_args={'sslmode':'require'})
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [2]:
def vguh(x):
    query = x
    return pd.io.sql.read_sql(sql=text(query), con = con)

## Количество книг вышло после 1 января 2000 года

In [3]:
print('Вывод: \nКоличество уникальных книг, вышедших после 1 января 2000 года =',vguh('''SELECT COUNT(*)
                                                                                 FROM books
                                                                                 WHERE publication_date > \'2000-01-01\'''').values[0][0])

Вывод: 
Количество уникальных книг, вышедших после 1 января 2000 года = 819


## Для каждой книги количество обзоров и среднюю оценку

In [4]:
vguh('''SELECT 
    b.book_id,
    b.title,
    r.mean_rating, 
    rev.total_reviews
    FROM 
    books b
    LEFT JOIN 
        (SELECT 
             book_id, 
             AVG(rating) as mean_rating 
         FROM 
             ratings 
         GROUP BY book_id
        ) r 
    ON b.book_id = r.book_id
    LEFT JOIN 
        (SELECT 
             book_id, 
             COUNT(review_id) as total_reviews 
         FROM 
             reviews 
         GROUP BY book_id
        ) rev 
    ON b.book_id = rev.book_id''')

,book_id,title,mean_rating,total_reviews
0,652,The Body in the Library (Miss Marple #3),4.500000,2.0
1,273,Galápagos,4.500000,2.0
2,51,A Tree Grows in Brooklyn,4.250000,5.0
3,951,Undaunted Courage: The Pioneering First Missio...,4.000000,2.0
4,839,The Prophet,4.285714,4.0
...,...,...,...,...
995,64,Alice in Wonderland,4.230769,4.0
996,55,A Woman of Substance (Emma Harte Saga #1),5.000000,2.0
997,148,Christine,3.428571,3.0
998,790,The Magicians' Guild (Black Magician Trilogy #1),3.500000,2.0


In [5]:
vguh('''SELECT 
    b.book_id,
    b.title,
    r.mean_rating, 
    rev.total_reviews
    FROM 
    books b
    LEFT JOIN 
        (SELECT 
             book_id, 
             AVG(rating) as mean_rating 
         FROM 
             ratings 
         GROUP BY book_id
        ) r 
    ON b.book_id = r.book_id
    LEFT JOIN 
        (SELECT 
             book_id, 
             COUNT(review_id) as total_reviews 
         FROM 
             reviews 
         GROUP BY book_id
        ) rev 
    ON b.book_id = rev.book_id
    WHERE rev.total_reviews IS NULL''')

,book_id,title,mean_rating,total_reviews
0,387,Leonardo's Notebooks,4.000000,None
1,191,Disney's Beauty and the Beast (A Little Golden...,4.000000,None
2,672,The Cat in the Hat and Other Dr. Seuss Favorites,5.000000,None
3,808,The Natural Way to Draw,3.000000,None
4,83,Anne Rice's The Vampire Lestat: A Graphic Novel,3.666667,None
5,221,Essential Tales and Poems,4.000000,None


### Вывод:
- Выведена таблица, в которой собры id, названия книг, средние оценки, количество обзоров
- Также выведена таблица с книгами на которые не было обзоров - их 6 

## Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так исключатся из анализа брошюры

In [6]:
top_publisher = vguh('''SELECT p_bs.publisher_id, p.publisher, p_bs.books
        FROM publishers p
        INNER JOIN
            (SELECT publisher_id, COUNT(*) AS books 
            FROM books 
            WHERE num_pages > 50 
            GROUP BY publisher_id
            ORDER BY COUNT(*) DESC
            LIMIT 1) as p_bs
        ON p.publisher_id = p_bs.publisher_id
        ''')

In [7]:
print('Вывод: \nИздательство с наибольшим количеством книг, страниц у которых больше 50.\nID:',top_publisher['publisher_id'][0],'\nНазвание:',top_publisher['publisher'][0],'\nКоличество книг:',top_publisher['books'][0])

Вывод: 
Издательство с наибольшим количеством книг, страниц у которых больше 50.
ID: 212 
Название: Penguin Books 
Количество книг: 42


## Автор с самой высокой средней оценкой книг — учитываются только книги с 50 и более оценками

In [8]:
task_3 = vguh('''SELECT au.author_id, au.author, B.avg
FROM authors AS au
INNER JOIN
    (SELECT author_id, avg
    FROM (
        SELECT author_id, AVG(t_p_b.avg) 
        FROM books AS b 
        RIGHT JOIN (
            SELECT book_id, AVG(rating) 
            FROM ratings 
            GROUP BY book_id 
            HAVING COUNT(rating_id)>49
            ) AS t_p_b 
        ON b.book_id = t_p_b.book_id 
        GROUP BY author_id 
        ) A 
    ORDER BY A.avg DESC
    LIMIT 1
    ) AS B
ON au.author_id = B.author_id''')[['author','avg']]

In [9]:
print('Вывод: \nАвтор с самой высокой средней оценкой книг — учитываются только книги с 50 и более оценками\nИмя:', task_3.values[0][0],'\nСредняя оценка:',task_3.values[0][1])

Вывод: 
Автор с самой высокой средней оценкой книг — учитываются только книги с 50 и более оценками
Имя: J.K. Rowling/Mary GrandPré 
Средняя оценка: 4.283844038868471


## Среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [10]:
print('Вывод: \nСреднее количество обзоров от пользователей, которые поставили больше 48 оценок =',vguh('''SELECT AVG(count)
        FROM (SELECT COUNT(text)
        FROM reviews
        WHERE username IN (SELECT username
        FROM ratings
        GROUP BY username
        HAVING COUNT(rating_id) > 48)
        GROUP BY username) AS A''').values[0][0])

Вывод: 
Среднее количество обзоров от пользователей, которые поставили больше 48 оценок = 24.0


## Вывод:
1. С 2000 года было выпущено 819 уникальных книг, что говорит о продолжающемся интересе к изданию книг и поддержании культуры чтения.
2. Для каждой книги были вычислены количество обзоров и средний рейтинг. Это может помочь определить, какие книги наиболее популярны среди читателей, и какие из них получают наиболее высокие оценки.
3. Издательство "Penguin Books" выпустило наибольшее количество книг (42), страниц которых больше 50. Это подчеркивает активность этого издательства и его преданность выпуску качественных, содержательных произведений.
4. Автор с самой высокой средней оценкой - "J.K. Rowling/Mary GrandPré", учитывая только книги с 50 и более оценками. Средний рейтинг составляет примерно 4.28, что является довольно высоким показателем и говорит о высоком качестве ее работ.
5. Пользователи, которые оставили более 48 оценок, в среднем пишут 24 обзора. Это показывает, что самые активные пользователи часто делятся своим мнением о прочитанных книгах, что может быть полезно для других читателей при выборе книг.